In [41]:
import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import speech_recognition as sr
import pyttsx3
import requests
import json
from bs4 import BeautifulSoup as bs
from difflib import SequenceMatcher
import subprocess
import random
import openai
import re
from time import sleep
import pyautogui as pg


In [42]:

steam_name = "yusufwadi"
openai.api_key_path = "key.txt"
cur_model = "text-davinci-002"
intents_model = "curie:ft-valley:intents-v3-2022-09-07-21-13-43"

r3 = sr.Recognizer()
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)


In [43]:
class person:
    name = ''

    def setName(self, name):
        self.name = name


person_obj = person()
person_obj.name = "Wadi"


In [44]:

def speak(audio_string):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    engine.say(audio_string)
    engine.runAndWait()


In [45]:

def listening(tts=""):
    with sr.Microphone() as source:
        r3 = sr.Recognizer()
        # r3.adjust_for_ambient_noise(source,duration=0.2)
        speak(tts)
        audio = r3.listen(source)
        voice = ""
        if audio != None:
            speak("Processing...")
        try:
            voice = r3.recognize_google(audio)  # convert audio to text
        except sr.UnknownValueError:  # error: recognizer does not understand
            print("...")

        except sr.RequestError:
            speak('Sorry, the service is down')
        print(f">> {voice.lower()}")
        return voice


In [46]:

def callGPT(prompt, tokens, model=cur_model):

    completion = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=0,
        frequency_penalty=0.5,
        max_tokens=tokens,

    )

    response = " ".join(re.findall("[a-zA-Z]+", completion.choices[0].text))
    response = str(response)

    return response


In [47]:
def smartListen(command):
    r3 = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r3.listen(source)
        if audio != None:
            speak("Processing...")
        try:
            keyword = r3.recognize_google(audio)  # convert audio to text
        except sr.UnknownValueError:  # error: recognizer does not understand
            print("...")
        except sr.RequestError:
            speak('Sorry, the service is down')

        print(f">> {keyword.lower()}")

        YOUR_PROMPT = " " + '"' + keyword + '"'

        response = callGPT(YOUR_PROMPT, 20)
        print(response)
        return response


In [48]:
def smartSearch(listen=False, keyword=""):
    if listen == True:
        r3 = sr.Recognizer()
        with sr.Microphone() as source:
            audio = r3.listen(source, phrase_time_limit=4)
            keyword = r3.recognize_google(audio)
    else:

        YOUR_PROMPT = f"Extract a search query from the following prompt: '{keyword}'"

        response = callGPT(YOUR_PROMPT, 20)

        return response


In [49]:


def getGame(response):

    URL = "https://steamcommunity.com/id/" + steam_name + "/games?tab=all&xml=1"
    data = requests.get(url=URL)
    soup = bs(data.text, 'xml')
    names = soup.find_all("name")
    response = response.lower()

    for name in names:

        if ((SequenceMatcher(a=response, b=name.text.lower()).ratio()) > 0.70):
            engine.say("Now launching... " + name.text)
            engine.runAndWait()
            print(name.text)
            print(name.parent.find("appID").text)

            return name.parent.find("appID").text
    return ""


In [50]:

# command functions


def greetings():
    greetings = [f"hey, how can I help you {person_obj.name}", f"hey, what's up? {person_obj.name}",
                 f"I'm listening {person_obj.name}", f"how can I help you? {person_obj.name}", f"hello {person_obj.name}"]
    greet = greetings[random.randint(0, len(greetings)-1)]
    speak(greet)

    return False


In [51]:

def goodbye():
    
    speak("going offline")

    return True


In [52]:

def play(command):

    prompt = f"Extract the name of the game from the prompt: {command}"

    response = callGPT(prompt, 20)

    gameID = getGame(response)
    try:
        subprocess.call([r'steam.bat', gameID])
    except TypeError:
        print("GAME NOT FOUND")
    return False


In [53]:

def search(command):
    return False


In [54]:

def time():
    e = datetime.datetime.now()
    speak("The time is now: = %s:%s" % (e.hour, e.minute))
    return False


In [55]:

def to_do(command):
    speak("THIS IS A TODO LIST")
    return False
            
#############

In [56]:
def open(command):
    if 'youtube' in command:

        response = smartSearch(keyword=command)
        say = f"Here is what I found for {response}"
        speak(say)
        driver = runWeb()
        driver.get(f'https://www.youtube.com/results?search_query={response}')
        driver.maximize_window()

    elif 'google' in command:
        driver = runWeb()
        driver.get('https://www.google.com/')
        driver.maximize_window()
        response = smartSearch()

        elem = driver.find_element('xpath', '//input[@class="gLFyf gsfi"]')
        elem.click()
        elem.send_keys(response, Keys.RETURN)


In [57]:
def close(command):

    if "exit app" in command:
        pg.hotkey('alt', 'f4')
        
    

In [58]:
def intents(command):
    print(command)
    completion = openai.Completion.create(
        model=intents_model,
        prompt=command,
        temperature=0,
        frequency_penalty=0.5,
        max_tokens=10,
        stop=["->"]
    )

    cleanComplete = completion.choices[0].text.split()[0]
    print(cleanComplete)
    return cleanComplete


In [59]:
def controlCenter(command):

    if command != "":
        command = command.lower()
        intent = intents(command + "->")
    else:
        return False

    match intent:
        case "greeting":
            return greetings()
        case "play":
            return play(command)
        case "search":
            return search(command)
        case "todo":
            return to_do(command)
        case "time":
            return time()
        case "goodbye":
            return goodbye()
        case "open":
            return open(command)
        case "close":
            return close(command)
        case _:
            print("command not recognized, try again")


In [60]:

done = False

speak("I am Idaw. What can i help with today sir?")
while not done:
    message = listening()
    done = controlCenter(message)


>> what time is it
what time is it->
time
